# **Proyecto Final**

\u003ca href=\"https://colab.research.google.com/github/Mario-Carmona/ProyectoFinal-VC/blob/main/ProyectoFinal.ipynb#scrollTo=hZxNTZOA2V-B\" target=\"_parent\"\u003e\u003cimg src=\"https://colab.research.google.com/assets/colab-badge.svg\" alt=\"Open In Colab\"/\u003e\u003c/a\u003e\n